In [1]:
using Random, Distributions
using LinearAlgebra, Statistics
using CSV, DataFrames
using Plots
using GLM, Econometrics

In [2]:
df = DataFrame(CSV.File("/Users/yinyin/Documents/GitHub/IO_spring/autodata.csv",delim=','))
a = 0.5
b = 1.5;

In [3]:
rename!(df,:"Air Conditioning" => :AC)
rename!(df,:"Weight of Car" => :Weight)
df.FirmID = df."Firm ID (there is a different number for each firm)"
df.p = df."Price(\$1000) (list price)"
select!(df, Not(:"Firm ID (there is a different number for each firm)"))
select!(df, Not(:"Price(\$1000) (list price)"))

#rank products by price (quality)
df = sort(df,:p) 

,Name,AC,Weight,Horsepower,Quantity,FirmID,p
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,YGGVPL,0,1870,67,6359,23,4435
2,SBJUST,0,1820,73,14363,5,5866
3,HYEXCE,0,2040,81,100590,21,5899
4,MTPREC,0,2336,76,4665,10,5899
5,GOMETR,0,1620,55,52409,19,5995
6,FDFEST,0,1785,63,52520,18,6319
7,TYTERC,0,2020,78,90808,1,6488
8,MZ323,0,2238,82,22977,4,6599
9,HDCIVI,0,2127,70,187240,3,6635


In [4]:
# Calculate market share, assume outside good account for 30%
df.s = 0.3*df.Quantity./sum(df.Quantity);

In [5]:
# Calculate \Delta recursively
N = length(df.p)
df.Δ = zeros(N)
df.Δ[N]= (b-a)*(1-df.s[N])+a
for i in 1:(N-1)
    df.Δ[N-i] = df.Δ[N-i+1] - df.s[N-i]*(b-a)
end

In [6]:
# Calculate quality recursively
df.δ = zeros(N)
df.δ[1] = df.p[1]/df.Δ[1]
for i in 2:N
    df.δ[i] = df.δ[i-1] + (df.p[i]-df.p[i-1])/df.Δ[i]
end

In [7]:
d1 = lm(@formula( δ ~ AC + Horsepower + Weight), df) # Without firm dummy

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,CholeskyPivoted{Float64,Array{Float64,2}}}},Array{Float64,2}}

δ ~ 1 + AC + Horsepower + Weight

Coefficients:
────────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)     Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────────
(Intercept)  -5956.48      3016.62     -1.97    0.0505  -11925.8        12.8676
AC            3001.16      1399.09      2.15    0.0338     232.618    5769.7
Horsepower     118.603       15.6951    7.56    <1e-11      87.5455    149.661
Weight           0.935137     1.37158   0.68    0.4966      -1.77897     3.64925
────────────────────────────────────────────────────────────────────────────────

In [8]:
d2 = lm(@formula( δ ~ AC + Horsepower + Weight + FirmID), df, contrasts = Dict(:FirmID => DummyCoding())) # With firm dummy

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,CholeskyPivoted{Float64,Array{Float64,2}}}},Array{Float64,2}}

δ ~ 1 + AC + Horsepower + Weight + FirmID

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)      Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)  -8496.01      2688.87    -3.16    0.0020  -13825.8       -3166.2
AC             637.361     1184.67     0.54    0.5917   -1710.86       2985.58
Horsepower      96.1109      14.6762   6.55    <1e-8       67.0202      125.202
Weight           2.54786      1.1854   2.15    0.0338       0.198183      4.89754
FirmID: 2     -263.6       1631.93    -0.16    0.8720   -3498.36       2971.16
FirmID: 3     1231.37      1883.31     0.65    0.5146   -2501.67       4964.4
FirmID: 4     1459.65      1886.88     0.77    0.4409   -2

In [9]:
# R^2
r2(d1)
r2(d2)

0.8702493910564055

In [10]:
#### Marginal Cost Pricing ###
df.xi = df.δ - predict(d2); # instruments for p - demand shifter
model_c = fit(EconometricModel,@formula(Quantity ~ (p ~ xi) + AC + Horsepower + Weight + FirmID),df,contrasts = Dict(:FirmID => DummyCoding()))

Continuous Response Model
Number of observations: 131
Null Loglikelihood: -1655.63
Loglikelihood: -1619.94
R-squared: NaN
LR Test: 71.38 ∼ χ²(23) ⟹  Pr > χ² = 0.0000
Formula: Quantity ~ 1 + (p ~ xi) + AC + Horsepower + Weight + FirmID
Variance Covariance Estimator: OIM
──────────────────────────────────────────────────────────────────────────────────────────────
                     PE           SE        t-value  Pr > |t|            2.50%          97.50%
──────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)    45714.5      52799.3       0.865816     0.3885   -58965.3             1.50394e5
AC            -49257.6      22128.6      -2.22596      0.0281   -93129.8         -5385.39
Horsepower      -159.963      322.39     -0.49618      0.6208     -799.133         479.206
Weight            44.7785      22.4916    1.9909       0.0491        0.186689       89.3703
FirmID: 2     -39762.0      30453.3      -1.30567      0.1945       -1.00139e

In [11]:
#### Firm Pricing #### 

# Cooperation structure
H = zeros(N,N)
for i in 1:N
    H[i,i]=1
end
for i in 1:(N-1)
    if (df.FirmID[i] == df.FirmID[i+1]) 
        H[i,i+1]=1
        H[i+1,i]=1
    end
end

In [12]:
T1 = zeros(N,N)
T1[1,1] = -(1/(df.δ[2]-df.δ[1])+1/df.δ[1])/(b-a)
T1[1,2] = (H[1,2]/(df.δ[2]-df.δ[1]))/(b-a)
T1[N,N] = -(1/(df.δ[N]-df.δ[N-1]))/(b-a)
T1[N,N-1] = (H[N,N-1]/(df.δ[N]-df.δ[N-1]))/(b-a)

for j in 2:(N-1)
    x = min(1/(df.δ[j+1]-df.δ[j]),1000)
    y = min(1/(df.δ[j]-df.δ[j-1]),1000)
    T1[j,j] = -(x+y)/(b-a)
    if H[j,j+1]==1
        T1[j,j+1] = min((1/(df.δ[j+1]-df.δ[j]))/(b-a),1000)
    end
    if H[j,j-1]==1
        T1[j,j-1] = min((1/(df.δ[j]-df.δ[j-1]))/(b-a),1000) 
    end
end

df.x1 = df.p + inv(T1)*df.s
model1 = fit(EconometricModel,@formula(Quantity ~ (x1 ~ xi) + AC + Horsepower + Weight + FirmID),df,contrasts = Dict(:FirmID => DummyCoding()))

Continuous Response Model
Number of observations: 131
Null Loglikelihood: -1655.63
Loglikelihood: -1619.94
R-squared: NaN
LR Test: 71.38 ∼ χ²(23) ⟹  Pr > χ² = 0.0000
Formula: Quantity ~ 1 + (x1 ~ xi) + AC + Horsepower + Weight + FirmID
Variance Covariance Estimator: OIM
──────────────────────────────────────────────────────────────────────────────────────────────
                      PE           SE        t-value  Pr > |t|           2.50%          97.50%
──────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)   45715.4        52799.0       0.865839     0.3885  -58963.7             1.50394e5
AC           -49257.4        22128.6      -2.22596      0.0281  -93129.4         -5385.35
Horsepower     -159.972        322.387    -0.49621      0.6208    -799.134         479.191
Weight           44.7784        22.4915    1.9909       0.0491       0.186781       89.37
FirmID: 2    -39761.7        30453.2      -1.30567      0.1945      -1.00138e5

In [18]:
#### Perfect Collusion #### 

T2 = zeros(N,N)
T2[1,1] = -(1/(df.δ[2]-df.δ[1])+1/df.δ[1])/(b-a)
T2[1,2] = (1/(df.δ[2]-df.δ[1]))/(b-a)
T2[N,N] = -(1/(df.δ[N]-df.δ[N-1]))/(b-a)
T2[N,N-1] = (1/(df.δ[N]-df.δ[N-1]))/(b-a)

for j in 2:(N-1)
    x = min(1/(df.δ[j+1]-df.δ[j]),1000)
    y = min(1/(df.δ[j]-df.δ[j-1]),1000)
    T2[j,j] = -(x+y)/(b-a)
    T2[j,j+1] = min((1/(df.δ[j+1]-df.δ[j]))/(b-a),1000)
    T2[j,j-1] = min((1/(df.δ[j]-df.δ[j-1]))/(b-a),1000) 
end

df.x2 = df.p + inv(T2)*df.s
model2 = fit(EconometricModel,@formula(Quantity ~ (x2 ~ xi) + AC + Horsepower + Weight + FirmID),df,contrasts = Dict(:FirmID => DummyCoding()))


Continuous Response Model
Number of observations: 131
Null Loglikelihood: -1655.63
Loglikelihood: -1620.06
R-squared: NaN
LR Test: 71.15 ∼ χ²(23) ⟹  Pr > χ² = 0.0000
Formula: Quantity ~ 1 + (x2 ~ xi) + AC + Horsepower + Weight + FirmID
Variance Covariance Estimator: OIM
──────────────────────────────────────────────────────────────────────────────────────────────
                      PE           SE       t-value  Pr > |t|            2.50%          97.50%
──────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)   43194.2        53238.4       0.811336    0.4190   -62356.1             1.48745e5
AC           -49477.1        22143.3      -2.2344      0.0276   -93378.4         -5575.78
Horsepower     -165.039        321.383    -0.513526    0.6087     -802.212         472.135
Weight           43.7454        22.426     1.95066     0.0537       -0.716271       88.2071
FirmID: 2    -39800.2        30480.3      -1.30577     0.1945       -1.0023e